In [1]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [2]:
from dotenv import load_dotenv
import os 

load_dotenv(override=True)

True

In [3]:
end_date = datetime.today() #--- made on 17.5.2025
start_date = end_date - timedelta(days=365 * 7)
start_str = start_date.strftime('%Y-%m-%d')
end_str = end_date.strftime('%Y-%m-%d')

In [4]:
def fetch_fred_series(series_id : str) -> pd.DataFrame:
    url = f'https://api.stlouisfed.org/fred/series/observations'

    params = {
        'series_id' : series_id,
        'api_key' : os.environ['FRED_API_KEY'],
        'file_type' : 'json',
        'observation_start' : start_str,
        'observation_end' : end_str
        
    }

    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data['observations'])
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace = True)
    df['value'] = df['value'].replace('.', np.nan).astype(float)
    df.rename(columns={'value':series_id}, inplace=True)

    return df

In [5]:
df = fetch_fred_series('CPIAUCSL')

print(len(df))

print(df.head())

84
           realtime_start realtime_end  CPIAUCSL
date                                            
2018-05-01     2025-05-17   2025-05-17   250.792
2018-06-01     2025-05-17   2025-05-17   251.018
2018-07-01     2025-05-17   2025-05-17   251.214
2018-08-01     2025-05-17   2025-05-17   251.663
2018-09-01     2025-05-17   2025-05-17   252.182


In [6]:
path = r'C:\Users\User\Desktop\🪙gold_iq\backend\data_collector\datasets2\inflation.csv'

df.to_csv(path)